In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# #read csv
data = pd.read_csv("/content/drive/My Drive/dit/HousingData.csv")

data_backup = data.copy()

In [ ]:

y = data['Median House Value']
X = data.drop('Median House Value', axis = 1)

data['Bedrooms Per Room'] = data['AveRooms'] / data['AveBedrms']
data['Occup Per Population'] = data['Population'] / data['AveOccup']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.28, random_state=42)


In [ ]:
#Linear Regression
reg = LinearRegression()

In [ ]:
reg = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
score = cross_val_score(reg , X_train, y_train, scoring = "neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-score)
tree_rmse_scores

array([0.80025744, 0.7239693 , 0.73154668, 0.6896349 , 0.76706543,
       0.70252619, 0.71325932, 0.75547639, 0.70501564, 1.9258619 ])

In [ ]:
param_grid = [
{'fit_intercept': [False, True], 'normalize': [True, False]},
]

grid_search = GridSearchCV(reg, param_grid, cv=5,scoring='neg_mean_squared_error',
return_train_score=True)


grid_search.fit(X_train, y_train)

grid_search.best_estimator_


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
reg = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

# #Plot

# plt.scatter(X_test, y_test,color='g')
# plt.plot(X_test, y_pred, color='k')

# plt.show()
np.mean((y_pred - y_test)**2)


0.543724164543471

In [ ]:
# Stochastic Gradient Descent
reg = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

# plt.scatter(X_test['MedInc'], y_test,color='g')
# plt.plot(X_test['MedInc'], y_pred,color='k')

# plt.show()

In [ ]:
# function to compute predictions created for the current dataset
# where y = w0 + w1*x1 +...+w8*x8 
def predict(X, y, theta):
  pred = np.dot(X_train, theta[0:8]) + theta[8]
  return pred
  
# function to compute gradient of error 
def gradient(X, y, theta): 
    h = np.dot(X, theta) 
    type(h)
    type(y)
    grad = (2/X.shape[0])*np.dot(X.transpose(), (h - y))
    # grad = 10
    return grad 
  
# function to compute MSE
def cost(X, y, theta): 
    h = np.dot(X, theta) 
    J = np.dot((h - y).transpose(), (h - y)) 
    J /= X.shape[0]
    return J[0] 
  
# function to perform mini-batch gradient descent
def gradientDescent(X, y, learning_rate = 0.1, steps=50): 
    X = np.c_[ X, np.ones(X.shape[0]) ] #add a column of ones to X for the bias term\
    theta = np.zeros((X.shape[1], 1)) #create inital weights w0, w1..wd
    
    error_list = []  
    for s in range(steps):
      theta = theta - learning_rate * gradient(X, y, theta)
      error_list.append(cost(X, y, theta)) #useful for plotting changes when using different batch sizes
  
    return theta, error_list 



theta, errors = gradientDescent(X_train, y_train)
predict(X_test, y_test, theta)

Exception: ignored

Exception: ignored